## Only get the lever Press time and add that to the df that we already have for the trials

In [49]:
%reload_ext autoreload
#%autoreload 2
%matplotlib inline
%run ../setup_project.py
%run ../generic_plot_functions.py
import os.path
from spikeA.Dat_file_reader import Dat_file_reader
from scipy.signal import welch
from scipy.ndimage import gaussian_filter1d
import numpy as np
from scipy.signal import find_peaks, peak_widths, peak_prominences
from scipy import integrate
from scipy.ndimage import gaussian_filter1d 
import os

projectName, dataPath, dlcModelPath, myProject, sSessions = setup_project_session_lists(projectName="autopi_inter",
                                                                dataPath="/adata/projects/autopi_inter/",dlcModelPath="/adata/models")


creating myProject, an autopipy.project object
Project name: autopi_inter
dataPath: /adata/projects/autopi_inter/
dlcModelPath: /adata/models
Reading /adata/projects/autopi_inter//sessionList
We have 180 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


In [120]:

prepareSessionsForSpatialAnalysisProject(sSessions,myProject.sessionList) ### sSeslist >>>> sSessions

Loading Animal_pose and Spike_train, sSes.ap and sSes.cg
Will load .pose.npy


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [04:32<00:00,  1.52s/it]


Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [137]:
def get_leverPress_time(ses, sSes):
    ses.extractTrialFeatures()
    
    lever_times = [(trial.leverPress['time'].to_list()[0], trial.name)
                   for trial in ses.trialList
                   if isinstance(trial.leverPress, pd.DataFrame) and len(trial.leverPress['time']) != 0]
    
    levertimedf = pd.DataFrame(lever_times, columns=['levertimeRos', 'name'])
    
    # Calculate LeverPressinRestime for each row
    leverPressTime = levertimedf['levertimeRos'].values
    resRosTime = np.vstack((sSes.ap.pose[:, 0], sSes.ap.pose[:, -1])).T
    levertimedf['levertimeRes'] = RosToResTime(rosTimes=leverPressTime, resRosTime=resRosTime)
    
    return levertimedf


In [138]:
#LeverPressinRestime= RosToResTime(rosTimes=leverPressTime, resRosTime=np.vstack((sSes.ap.pose[:,0],sSes.ap.pose[:,-1])).T)

In [139]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)


In [ ]:
ses= myProject.sessionList[4]
levertimedf= get_leverPress_time(ses)

In [ ]:
%%time
LeverPress=[]

for ses,sSes in tqdm(zip(myProject.sessionList, sSessions)):
    
    df= get_leverPress_time(ses, sSes)
    LeverPress.append(df)
    
leverTime = pd.concat(LeverPress, ignore_index=True)

In [148]:
len(leverTime),len(sSessions)

(20661, 180)

In [150]:
leverTime.shape

(20661, 3)

In [161]:

fn ='leverTimes.csv'
# Save the DataFrame to a CSV file
leverTime.to_csv(fn, index=False)

In [152]:
fn ='leverTimes.cv'
leverTime= pd.read_csv(fn)

In [153]:
levertimedf

,levertimeRos,name,levertimeRes
0,1.638958e+09,mn8578-08122021-0110_1,3042.968709
1,1.638958e+09,mn8578-08122021-0110_2,3127.853149
2,1.638958e+09,mn8578-08122021-0110_3,3169.631187
3,1.638958e+09,mn8578-08122021-0110_4,3253.638913
4,1.638958e+09,mn8578-08122021-0110_5,3345.494984
...,...,...,...
93,1.638964e+09,mn8578-08122021-0110_94,9028.580759
94,1.638964e+09,mn8578-08122021-0110_95,9363.300316
95,1.638965e+09,mn8578-08122021-0110_96,9919.199525
96,1.638965e+09,mn8578-08122021-0110_97,9970.495351


In [154]:
res = pd.read_csv('behavior_180_EastReferenceQuadrant.csv')

In [155]:
res

,sessionName,name,valid,trialNo,startTime,endTime,startTimeWS,endTimeWS,duration,light,...,startPositionHoming_x,startPositionHoming_y,homingAngleAtPeriphery,homingErrorAtPeriphery,homingErrorAtPeripheryLever,homingErrorAtPeripheryLeverAbsolute,leverX,leverY,subject,date
0,mn8578-30112021-0107,mn8578-30112021-0107_1,True,1,1.638266e+09,1.638266e+09,3.330952,43.437132,40.106180,light,...,-2.366743,-11.459763,-1.610825,-0.083860,-0.083781,0.083781,1.643595,1.114050,mn8578,2021-11-30 01:07:00
1,mn8578-30112021-0107,mn8578-30112021-0107_2,True,2,1.638266e+09,1.638266e+09,51.433192,68.696401,17.263209,light,...,-3.288517,-14.331650,-1.633013,-0.106048,-0.114553,0.114553,1.881237,-2.031261,mn8578,2021-11-30 01:07:00
2,mn8578-30112021-0107,mn8578-30112021-0107_3,True,3,1.638266e+09,1.638266e+09,73.689115,105.833658,32.144543,light,...,-3.894192,-13.361742,-1.578244,-0.051279,-0.057984,0.057984,1.876152,-2.032935,mn8578,2021-11-30 01:07:00
3,mn8578-30112021-0107,mn8578-30112021-0107_4,True,4,1.638266e+09,1.638266e+09,118.842181,153.795247,34.953066,light,...,5.344472,-9.157124,-1.565123,-0.038158,-0.034961,0.034961,-1.429608,1.026417,mn8578,2021-11-30 01:07:00
4,mn8578-30112021-0107,mn8578-30112021-0107_5,True,5,1.638266e+09,1.638266e+09,175.030779,214.734324,39.703545,light,...,-23.345750,-12.715746,-1.754851,-0.227887,-0.145241,0.145241,-20.592588,1.260800,mn8578,2021-11-30 01:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20697,jp499-25052023-0108,jp499-25052023-0108_111,True,111,1.685029e+09,1.685029e+09,6426.638292,6460.432821,33.794529,light,...,15.434140,-34.049265,-0.844326,0.802898,1.429400,1.429400,20.593755,-20.931358,jp499,2023-05-25 01:08:00
20698,jp499-25052023-0108,jp499-25052023-0108_112,True,112,1.685029e+09,1.685029e+09,6478.443990,6528.834271,50.390282,dark,...,6.785348,16.395909,1.745148,-2.890813,-2.601360,2.601360,-1.680767,30.069567,jp499,2023-05-25 01:08:00
20699,jp499-25052023-0108,jp499-25052023-0108_113,True,113,1.685029e+09,1.685029e+09,6555.648851,6620.684408,65.035557,light,...,-2.724584,-29.429458,-1.619052,0.028171,0.100612,0.100612,-14.415291,-18.097031,jp499,2023-05-25 01:08:00
20700,jp499-25052023-0108,jp499-25052023-0108_114,True,114,1.685029e+09,1.685029e+09,6625.678306,6648.324384,22.646078,dark,...,-19.918364,-30.822634,-2.124231,-0.477008,-0.836805,0.836805,-14.309954,-17.895829,jp499,2023-05-25 01:08:00


In [89]:
leverTime.shape, res.shape

((20661, 2), (20702, 30))

In [90]:
leverTime.keys()

Index(['levertimeRos', 'trial'], dtype='object')

In [75]:
res.keys()

Index(['sessionName', 'name', 'valid', 'trialNo', 'startTime', 'endTime',
       'startTimeWS', 'endTimeWS', 'duration', 'light', 'arenaRadiusCm',
       'nLeverPresses', 'nJourneys', 'searchLength', 'searchDuration',
       'searchMeanSpeed', 'searchMedianMVDeviationToTarget',
       'searchLastTargetToAnimalAngle', 'medianMVDeviationRoomReference',
       'headingError', 'startPositionHoming_x', 'startPositionHoming_y',
       'homingAngleAtPeriphery', 'homingErrorAtPeriphery',
       'homingErrorAtPeripheryLever', 'homingErrorAtPeripheryLeverAbsolute',
       'leverX', 'leverY', 'subject', 'date'],
      dtype='object')

In [102]:
leverTime

,levertimeRos,name
0,1.638266e+09,mn8578-30112021-0107_1
1,1.638266e+09,mn8578-30112021-0107_2
2,1.638266e+09,mn8578-30112021-0107_3
3,1.638266e+09,mn8578-30112021-0107_4
4,1.638266e+09,mn8578-30112021-0107_5
...,...,...
20656,1.685029e+09,jp499-25052023-0108_110
20657,1.685029e+09,jp499-25052023-0108_111
20658,1.685029e+09,jp499-25052023-0108_112
20659,1.685029e+09,jp499-25052023-0108_113


In [156]:
res2= res.merge(leverTime, on=['name'])


In [157]:
res.keys()

Index(['sessionName', 'name', 'valid', 'trialNo', 'startTime', 'endTime',
       'startTimeWS', 'endTimeWS', 'duration', 'light', 'arenaRadiusCm',
       'nLeverPresses', 'nJourneys', 'searchLength', 'searchDuration',
       'searchMeanSpeed', 'searchMedianMVDeviationToTarget',
       'searchLastTargetToAnimalAngle', 'medianMVDeviationRoomReference',
       'headingError', 'startPositionHoming_x', 'startPositionHoming_y',
       'homingAngleAtPeriphery', 'homingErrorAtPeriphery',
       'homingErrorAtPeripheryLever', 'homingErrorAtPeripheryLeverAbsolute',
       'leverX', 'leverY', 'subject', 'date'],
      dtype='object')

In [158]:
res2.keys()

Index(['sessionName', 'name', 'valid', 'trialNo', 'startTime', 'endTime',
       'startTimeWS', 'endTimeWS', 'duration', 'light', 'arenaRadiusCm',
       'nLeverPresses', 'nJourneys', 'searchLength', 'searchDuration',
       'searchMeanSpeed', 'searchMedianMVDeviationToTarget',
       'searchLastTargetToAnimalAngle', 'medianMVDeviationRoomReference',
       'headingError', 'startPositionHoming_x', 'startPositionHoming_y',
       'homingAngleAtPeriphery', 'homingErrorAtPeriphery',
       'homingErrorAtPeripheryLever', 'homingErrorAtPeripheryLeverAbsolute',
       'leverX', 'leverY', 'subject', 'date', 'levertimeRos', 'levertimeRes'],
      dtype='object')

## Add the leverpress res and ros time to the df we have 

In [160]:
fn ='behavior_180_v2.csv'
res2.to_csv(fn, index=False)